In [34]:
import os
import pandas as pd
from dotenv import load_dotenv
import requests_cache
from retry_requests import retry

def get_env_vars():
    """
    Loads latitude, longitude, and tilt from the env module.
    Raises an exception if any variable is missing.
    """
    try:
        # Load environment variables from .env file
        load_dotenv()
        lat = os.getenv('LATITUDE')
        lon = os.getenv('LONGITUDE')
        tilt = os.getenv('PANEL_TILT')

        if lat is None or lon is None or tilt is None:
            raise RuntimeError("Missing required environment variables: LAT, LON, or TILT.")

        return lat, lon, tilt
    
    except AttributeError as e:
        print(f"Error in getting environment variable:  {e}")
        raise RuntimeError(f"Missing required environment variable: {e}")

def fetch_weather_forecast():
    """
    Fetches weather forecast data from the Open-Meteo API,
    caches responses, and returns hourly forecast of 5 days.
    """
    lat, lon, tilt = get_env_vars()
    cache_dir = 'cache'
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, 'weather_forecast')
    cache_session = requests_cache.CachedSession(cache_path, expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)

    forecast_api_url = (
        "https://api.open-meteo.com/v1/forecast"
    )
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m,precipitation_probability,precipitation,cloud_cover,wind_speed_10m,wind_direction_10m,is_day,shortwave_radiation,global_tilted_irradiance",
        "timezone": "Asia/Kolkata",
        "tilt": tilt
    }

    try:
        response = retry_session.get(forecast_api_url, params=params)
        response.raise_for_status()
        data = response.json()

        if 'hourly' not in data or 'time' not in data['hourly']:
            raise ValueError("API response missing required 'hourly' or 'time' fields.")
        
        return data['hourly']
        
    except Exception as e:
        print(f"Error fetching weather forecast: {e}")
        raise RuntimeError(f"Failed to fetch weather forecast: {e}")

def process_weather_forecast(weather_forecast):
    """
    Formats the weather forecast into DataFrame to predict energy_usage by ML model
    """
    try:
        forecast_df = pd.DataFrame(weather_forecast)
        forecast_df['time'] = pd.to_datetime(forecast_df['time'])
        forecast_df.set_index('time', inplace=True)

        forecast_df.rename(columns={
            'temperature_2m': 'temperature',
            'relative_humidity_2m': 'humidity',
            'precipitation_probability': 'precipitation_prob',
            'wind_speed_10m': 'wind_speed',
            'wind_direction_10m': 'wind_direction',
            'shortwave_radiation': 'GHI',
            'global_tilted_irradiance': 'GTI',
        }, inplace=True)

        # float_cols = forecast_df.select_dtypes(include=['float64']).columns
        # int_cols = forecast_df.select_dtypes(include=['int64']).columns
        # forecast_df[float_cols] = forecast_df[float_cols].astype('float32')
        # forecast_df[int_cols] = forecast_df[int_cols].astype('int32')
        
        return forecast_df
    
    except Exception as e:
        print(f"Error in processing weather forecast:  {e}")
        raise RuntimeError(f"Failed to convert weather dictionary to DataFrame: {e}")

In [35]:
weather_forecast = fetch_weather_forecast()

forecast_df = process_weather_forecast(weather_forecast)
forecast_df.head()

,temperature,humidity,precipitation_prob,precipitation,cloud_cover,wind_speed,wind_direction,is_day,GHI,GTI
time,,,,,,,,,,
2025-05-19 00:00:00,28.2,79,35,0.0,75,5.7,252,0,0.0,0.0
2025-05-19 01:00:00,27.6,80,33,0.0,84,7.3,279,0,0.0,0.0
2025-05-19 02:00:00,26.8,81,33,1.4,100,6.6,283,0,0.0,0.0
2025-05-19 03:00:00,26.4,81,25,1.2,100,6.5,289,0,0.0,0.0
2025-05-19 04:00:00,26.1,82,18,0.8,100,3.4,302,0,0.0,0.0


In [36]:
forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 168 entries, 2025-05-19 00:00:00 to 2025-05-25 23:00:00
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         168 non-null    float64
 1   humidity            168 non-null    int64  
 2   precipitation_prob  168 non-null    int64  
 3   precipitation       168 non-null    float64
 4   cloud_cover         168 non-null    int64  
 5   wind_speed          168 non-null    float64
 6   wind_direction      168 non-null    int64  
 7   is_day              168 non-null    int64  
 8   GHI                 168 non-null    float64
 9   GTI                 168 non-null    float64
dtypes: float64(5), int64(5)
memory usage: 14.4 KB


In [41]:
import joblib

# Set up a cache directory for predictions
memory = joblib.Memory(location='cache/energy_predictions', verbose=0)

@memory.cache
def predict_energy_usage(weather_forecast):
    """
    Predicts energy usage based on weather forecast data with ML model.
    """
    try:
        model = joblib.load("C:/Users/vishw/Projects/Solaris/ml_models/household_energy_model.pkl")

        predictions = model.predict(weather_forecast)

        return predictions
    
    except Exception as e:
        print(f"Error in predicting energy demand:  {e}")
        raise RuntimeError(f"Failed to predict energy usage: {e}")

def fetch_energy_predictions(weather_forecast):
    """
    Fetchs the energy predictions made by ML model from forecast weather and combines into single DataFrame
    """
    try:
        prediction_df = weather_forecast[['temperature', 'humidity', 'precipitation', 'cloud_cover', 'is_day']].copy()
        
        predictions = predict_energy_usage(prediction_df)

        weather_forecast['energy_demand'] = predictions

        weather_forecast['energy_demand'] = weather_forecast['energy_demand'].round(2)

        return weather_forecast
    
    except Exception as e:
        print(f"Error in generating energy predictions:  {e}")
        raise RuntimeError(f"Failed to fetch energy predictions and merge forecast weather")

In [42]:
weather_data = fetch_weather_forecast()

forecast_data = process_weather_forecast(weather_data)

prediction_df = fetch_energy_predictions(forecast_data)

prediction_df.head()

,temperature,humidity,precipitation_prob,precipitation,cloud_cover,wind_speed,wind_direction,is_day,GHI,GTI,energy_demand
time,,,,,,,,,,,
2025-05-19 00:00:00,28.2,79,35,0.0,75,5.7,252,0,0.0,0.0,1.03
2025-05-19 01:00:00,27.6,80,33,0.0,84,7.3,279,0,0.0,0.0,1.39
2025-05-19 02:00:00,26.8,81,33,1.4,100,6.6,283,0,0.0,0.0,1.05
2025-05-19 03:00:00,26.4,81,25,1.2,100,6.5,289,0,0.0,0.0,1.08
2025-05-19 04:00:00,26.1,82,18,0.8,100,3.4,302,0,0.0,0.0,1.30


In [43]:
class BatteryModule:
    def __init__(self, capacity, degradation_rate, current_charge=None, total_charged=0, charge_cycles=2):
        self.max_capacity = capacity
        self.capacity = capacity

        self.current_charge = current_charge if current_charge is not None else capacity / 2
        self.battery_percentage = (self.current_charge / self.capacity) * 100

        self.max_charge_rate = 1.5
        self.max_discharge_rate = 2.0
        
        self.total_charged = total_charged
        self.charge_cycles = charge_cycles
        self.degradation_rate = degradation_rate
        self.previous_full_cycles = 0

    def charge(self, amount):
        if amount < 0:
            raise ValueError("Charge amount must be positive")
        new_charge = self.current_charge + amount
        if new_charge > self.capacity:
            self.current_charge = self.capacity
        else:
            self.current_charge = new_charge
        self.battery_percentage = (self.current_charge / self.capacity) * 100 if self.capacity > 0 else 0

        self.degrade_battery(amount)

    def discharge(self, amount):
        if amount < 0:
            raise ValueError("Discharge amount must be positive")
        new_charge = self.current_charge - amount
        if new_charge < 0:
            self.current_charge = 0
        else:
            self.current_charge = new_charge
        self.battery_percentage = (self.current_charge / self.capacity) * 100 if self.capacity > 0 else 0

    def degrade_battery(self, amount):
        self.total_charged += amount
        self.charge_cycles = self.total_charged / self.max_capacity

        full_cycles = int(self.charge_cycles)
        if full_cycles > self.previous_full_cycles:
            cycles_to_degrade = full_cycles - self.previous_full_cycles
            capacity_loss = cycles_to_degrade * self.degradation_rate
            self.capacity -= capacity_loss

            if self.capacity < 0.01:
                self.capacity = 0.01
            self.previous_full_cycles = full_cycles

In [47]:
import os
import json
import pandas as pd
from dotenv import load_dotenv

def load_battery_state(file_path='cache/battery_state.json'):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            battery_data = json.load(f)
        return battery_data
    else:
        return None  # Or defaults

def save_battery_state(battery, file_path='cache/battery_state.json'):
    battery_data = {
        'battery_capacity': battery.capacity,
        'battery_charge': battery.current_charge,
        'battery_percentage': (battery.current_charge / battery.capacity) * 100,
        'battery_charge_cycles': battery.charge_cycles,
        'total_battery_charged': battery.total_charged
    }
    with open(file_path, 'w') as f:
        json.dump(battery_data, f, indent=4)

def decision_making(testing = False):
    '''
    Make a decision on energy source selection and battery decision
    '''
    try:
        load_dotenv()
        panel_capacity = float(os.getenv('PANEL_CAPACITY'))
        panel_area = float(os.getenv('PANEL_AREA'))
        panel_efficiency = float(os.getenv('PANEL_EFFICIENCY'))
        panel_tilt = float(os.getenv('PANEL_TILT'))
        battery_capacity = float(os.getenv('BATTERY_CAPACITY'))
        battery_degradation_rate = float(os.getenv('DEGRADATION_RATE'))

        if testing:
            forecast_df = pd.read_csv("C:/Users/vishw/Projects/Solaris/datasets/weather_data.csv")
        else:
            weather_forecast = fetch_weather_forecast()
            forecast_df = process_weather_forecast(weather_forecast)
        
        forecast_data = fetch_energy_predictions(forecast_df)
        
        # Calculate solar output (capped at panel capacity)
        forecast_data['solar_output_calc'] = (forecast_data['GTI'] * panel_efficiency * panel_area) / 1000
        forecast_data['solar_output'] = forecast_data[['solar_output_calc']].clip(upper=panel_capacity)

        battery_state = load_battery_state()

        if battery_state:
            battery = BatteryModule(
                capacity=battery_state['battery_capacity'],
                degradation_rate=battery_degradation_rate,
                current_charge=battery_state['battery_charge'],
                charge_cycles=battery_state['battery_charge_cycles'],
                total_charged=battery_state['total_battery_charged']
            )
        else:
            battery = BatteryModule(battery_capacity, battery_degradation_rate)

        energy_demand = forecast_data['energy_demand'].iloc[0]
        solar_output = forecast_data['solar_output'].iloc[0]

        decisions = []
        battery_curr_capacity = []
        battery_levels = []
        battery_percentages = []
        battery_charge_rates = []
        battery_total_charged = []

         # Loop over each row of forecast data
        for _, row in forecast_data.iterrows():
            energy_demand = row['energy_demand']
            solar_output = row['solar_output']

            if energy_demand <= solar_output:
                # Use solar energy, charge battery with surplus
                decisions.append('solar')
                surplus_energy = solar_output - energy_demand
                if surplus_energy > battery.max_charge_rate:
                    battery.charge(battery.max_charge_rate)
                else:
                    battery.charge(surplus_energy)

            elif battery.current_charge >= (energy_demand - solar_output):
                # Use battery to meet the remaining demand
                decisions.append('battery')
                if (energy_demand - solar_output) > battery.max_discharge_rate:
                    battery.discharge(battery.max_discharge_rate)
                else:
                    battery.discharge(energy_demand - solar_output)

            else:
                # Use grid if solar + battery cannot meet demand
                decisions.append('grid')

            # Record battery state
            battery_curr_capacity.append(battery.capacity)
            battery_levels.append(battery.current_charge)
            battery_percentages.append(battery.battery_percentage)
            battery_charge_rates.append(battery.charge_cycles)
            battery_total_charged.append(battery.total_charged)

        # Add new columns to DataFrame
        forecast_data['decision'] = decisions
        forecast_data['battery_capacity'] = battery_curr_capacity
        forecast_data['battery_charge'] = battery_levels
        forecast_data['battery_percentage'] = battery_percentages
        forecast_data['battery_charge_cycles'] = battery_charge_rates
        forecast_data['total_battery_charged'] = battery_total_charged

        return forecast_data  # Optional: return for further use

    except Exception as e:
        print("Error making decision of energy source", {e})
        raise RuntimeError(f"Failed to make a decision at energy source selection: {e}")

In [48]:
forecast_df = decision_making()

In [49]:
forecast_df.head()

,temperature,humidity,precipitation_prob,precipitation,cloud_cover,wind_speed,wind_direction,is_day,GHI,GTI,energy_demand,solar_output_calc,solar_output,decision,battery_capacity,battery_charge,battery_percentage,battery_charge_cycles,total_battery_charged
time,,,,,,,,,,,,,,,,,,,
2025-05-19 00:00:00,28.2,79,35,0.0,75,5.7,252,0,0.0,0.0,1.03,0.0,0.0,battery,20.0,8.97,44.85,2.0,0.0
2025-05-19 01:00:00,27.6,80,33,0.0,84,7.3,279,0,0.0,0.0,1.39,0.0,0.0,battery,20.0,7.58,37.90,2.0,0.0
2025-05-19 02:00:00,26.8,81,33,1.4,100,6.6,283,0,0.0,0.0,1.05,0.0,0.0,battery,20.0,6.53,32.65,2.0,0.0
2025-05-19 03:00:00,26.4,81,25,1.2,100,6.5,289,0,0.0,0.0,1.08,0.0,0.0,battery,20.0,5.45,27.25,2.0,0.0
2025-05-19 04:00:00,26.1,82,18,0.8,100,3.4,302,0,0.0,0.0,1.30,0.0,0.0,battery,20.0,4.15,20.75,2.0,0.0


In [50]:
forecast_df.tail()

,temperature,humidity,precipitation_prob,precipitation,cloud_cover,wind_speed,wind_direction,is_day,GHI,GTI,energy_demand,solar_output_calc,solar_output,decision,battery_capacity,battery_charge,battery_percentage,battery_charge_cycles,total_battery_charged
time,,,,,,,,,,,,,,,,,,,
2025-05-25 19:00:00,30.3,59,20,0.0,58,13.1,238,0,8.0,4.0,1.08,0.0144,0.0144,battery,19.996,8.40968,42.056812,2.452698,49.05396
2025-05-25 20:00:00,29.2,64,21,0.0,56,12.2,236,0,0.0,0.0,1.17,0.0000,0.0000,battery,19.996,7.23968,36.205642,2.452698,49.05396
2025-05-25 21:00:00,28.6,67,22,0.0,64,11.9,235,0,0.0,0.0,1.16,0.0000,0.0000,battery,19.996,6.07968,30.404482,2.452698,49.05396
2025-05-25 22:00:00,28.2,68,23,0.0,71,12.1,233,0,0.0,0.0,0.95,0.0000,0.0000,battery,19.996,5.12968,25.653532,2.452698,49.05396
2025-05-25 23:00:00,27.8,69,24,0.0,79,12.4,234,0,0.0,0.0,0.98,0.0000,0.0000,battery,19.996,4.14968,20.752552,2.452698,49.05396


In [14]:
forecast_df.describe()

,temperature,humidity,precipitation_prob,precipitation,cloud_cover,wind_speed,wind_direction,is_day,GHI,GTI,energy_demand,solar_output_calc,solar_output,battery_capacity,battery_charge,battery_percentage,battery_charge_cycles,total_battery_charged
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,28.962502,68.767857,38.422619,0.235119,82.732143,7.626190,220.000000,0.541667,205.410721,195.642853,1.020629,0.704314,0.704314,19.998846,3.072127,15.361828,1.152637,20.909889
std,2.830198,13.983726,18.936862,0.591875,20.766300,3.986478,68.676365,0.499750,266.930786,257.035278,0.284468,0.925327,0.925327,0.001286,3.132420,15.663658,0.660581,13.531050
min,24.700001,39.000000,9.000000,0.000000,18.000000,0.700000,9.000000,0.000000,0.000000,0.000000,0.210478,0.000000,0.000000,19.996000,0.044156,0.220803,0.023078,0.000000
25%,26.600000,58.000000,24.000000,0.000000,64.000000,4.575000,219.000000,0.000000,0.000000,0.000000,0.824167,0.000000,0.000000,19.997999,0.300664,1.503322,0.700186,14.003721
50%,28.200001,72.000000,36.500000,0.000000,91.500000,7.300000,242.000000,1.000000,44.500000,34.200001,1.010302,0.123120,0.123120,20.000000,1.727473,8.637363,1.022504,18.512486
75%,31.299999,81.250000,46.000000,0.100000,100.000000,10.500000,262.250000,1.000000,393.250000,368.525009,1.187562,1.326690,1.326690,20.000000,5.488891,27.446510,1.858726,28.236460
max,35.299999,89.000000,85.000000,3.600000,100.000000,17.400000,342.000000,1.000000,913.000000,875.000000,2.186788,3.150000,3.150000,20.000000,10.679521,53.402943,2.453185,49.063702


In [52]:
forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 168 entries, 2025-05-19 00:00:00 to 2025-05-25 23:00:00
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   temperature            168 non-null    float64
 1   humidity               168 non-null    int64  
 2   precipitation_prob     168 non-null    int64  
 3   precipitation          168 non-null    float64
 4   cloud_cover            168 non-null    int64  
 5   wind_speed             168 non-null    float64
 6   wind_direction         168 non-null    int64  
 7   is_day                 168 non-null    int64  
 8   GHI                    168 non-null    float64
 9   GTI                    168 non-null    float64
 10  energy_demand          168 non-null    float32
 11  solar_output_calc      168 non-null    float64
 12  solar_output           168 non-null    float64
 13  decision               168 non-null    object 
 14  battery_capacity     